<h1><center>Predict the default of Credit Card holder</center></h1>
<h2><center>Group 1: 
Li, Qiao; Shu, Chuyun; Xia, Zhihang; Yu, Shuqi; Yuan, Serena; Zhang, Renyin </center></h2>

# 1. Introduction

# Context
Our study took payment data in October, 2005, from an
important bank (a cash and credit card issuer) in Taiwan
and the targets were credit card holders of the bank.
Among the total 25,000 observations, 5529 observations
(22.12%) are the cardholders with default payment. This
research employed a binary variable – default payment
(Yes = 1, No = 0), as the response variable.

# Content
Following 23 variables as explanatory variables:
 
 X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit.
 
 X2: Gender (1 = male; 2 = female).
 
 X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others).

 X4: Marital status (1 = married; 2 = single; 3 = others).
 
 X5: Age (year).
 
 X6–X11: History of past payment. We tracked the past monthly payment records (from April to September,2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August,2005;...;X11 = the repayment status in April, 2005.
The measurement scale for the repayment status is:
1 = pay duly; 1 = payment delay for one month;2 = payment delay for two months; ...; 8 = payment delay for eight months; 9 = payment delay for nine months and above.
 
 X12–X17: Amount of bill statement (NT dollar).(X12 = amount of bill statement in September,2005; X13 = amount of bill statement in August,2005 ;...; X17 = amount of bill statement in April,2005.)
 
 X18–X23: Amount of previous payment (NT dollar).(X18 = amount paid in September, 2005; X19 = amount paid in August, 2005;...;X23 = amount paid in April,2005.) 

# 2. Set up the environment
First, make sure you have python installed. 

In [ ]:
!python --version

If you don't have python installed - please follow this [link](https://realpython.com/installing-python/) for instructions.

Running the cell below will start the installation of required python libraries, assuming you have python already installed.


In [1]:
!pip install pandas
!pip install numpy 
!pip install seaborn
!pip install matplotlib
!pip install plotly
!pip install iplot

In [1]:
#import the librarys
import pandas as pd 
import smote_variants as sv
import imbalanced_databases as imbd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import plotly as py
import plotly.graph_objs as go
from plotly.offline import *
import seaborn as sns

%matplotlib inline
np.random.seed(2021)

ImportDataset

In [2]:
df = pd.read_excel("C:/Users/liqia/Spring2021-Project5-group-1-1-master/data/default of credit card clients_original.xls") 
df = df.iloc[1:, 1:]
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


# 3 Data Preprocessing
The data was randomly divided into two groups, one for model training and the other to validate the model.

In [3]:
#Creating the X and y variables
X = df.loc[:,'X1':'X23']
y = df.loc[:, 'Y'].astype('category')
print('good class: %d' % np.sum(y == 0))
print('bad class: %d' % np.sum(y == 1))

good class: 23364
bad class: 6636


In [4]:
X.shape,y.shape

((30000, 23), (30000,))

Data Preprocessing Imbalanced Dataset (SMOTE)-oversamplig

In [5]:
from imblearn.over_sampling import SMOTE
# transform the dataset
oversample = SMOTE()
X_balance, y_balance = oversample.fit_resample(X, y)

X.shape,y.shape

((30000, 23), (30000,))

Train-test split since we can see that the dataset is imbalanced, in this chunk we do an 80-20 train-test split.


In [6]:
#split train and test with size=0.20.

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_balance, y_balance,test_size=0.2,random_state=0)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((37382, 23), (37382,), (9346, 23), (9346,))

# 4. Models

# SVM

In [7]:

import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV,RepeatedStratifiedKFold,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
import scipy.io
import pickle
import os, sys
from scipy.spatial.distance import pdist
import time 
import xlsxwriter
from sklearn.metrics import accuracy_score, classification_report,make_scorer, confusion_matrix,roc_auc_score
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
#from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from sklearn import ensemble
from sklearn.svm import SVC
import random
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Input, Dropout
from keras.layers import BatchNormalization
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.utils import to_categorical

In [8]:
param_grid = {'C': [0.1,1, 10, 100, 1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel': ['rbf']} 
grid_svm = GridSearchCV(SVC(),param_grid,refit=True,verbose=3)
grid_svm.fit(X_train,y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time= 6.2min
[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time= 6.9min
[CV 3/5] END .....................C=0.1, gamma=1, kernel=rbf; total time= 5.9min
[CV 4/5] END .....................C=0.1, gamma=1, kernel=rbf; total time= 6.0min
[CV 5/5] END .....................C=0.1, gamma=1, kernel=rbf; total time= 5.4min
[CV 1/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time= 5.1min
[CV 2/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time= 4.9min
[CV 3/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time= 4.9min
[CV 4/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time= 4.9min
[CV 5/5] END ...................C=0.1, gamma=0.1, kernel=rbf; total time= 5.1min
[CV 1/5] END ..................C=0.1, gamma=0.01, kernel=rbf; total time= 6.4min
[CV 2/5] END ..................C=0.1, gamma=0.0

[CV 2/5] END ....................C=1000, gamma=1, kernel=rbf; total time=10.8min
[CV 3/5] END ....................C=1000, gamma=1, kernel=rbf; total time=11.6min
[CV 4/5] END ....................C=1000, gamma=1, kernel=rbf; total time=12.2min
[CV 5/5] END ....................C=1000, gamma=1, kernel=rbf; total time=19.7min
[CV 1/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=16.7min
[CV 2/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=18.1min
[CV 3/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=18.5min
[CV 4/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=19.3min
[CV 5/5] END ..................C=1000, gamma=0.1, kernel=rbf; total time=21.9min
[CV 1/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=16.7min
[CV 2/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=22.3min
[CV 3/5] END .................C=1000, gamma=0.01, kernel=rbf; total time=22.8min
[CV 4/5] END ...............

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [9]:
print(grid_svm.best_params_)
print(grid_svm.best_estimator_)

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
SVC(C=1000, gamma=0.0001)


In [10]:
#Train SVM using best parameters
svm_best = SVC(C=1000,gamma=0.0001,kernel='rbf',probability=True) 
start_time=time.time()
svm_best.fit(X_train, y_train)

print("Training  model takes %s seconds" % round((time.time() - start_time),3))
print('Accuracy of SVM on test set: {:.3f}'.format(svm_best.score(X_test,y_test)))

start = time.time()
svm_pred = svm_best.predict(X_test)
end = time.time()

svm_predprob = svm_best.predict_proba(X_test)[:,1]

print("Predicting test data takes %s seconds" % round((end - start),3))
print('Classification error rate:', np.mean(np.array(y_test)!= svm_pred))
print('Classification report \n', classification_report(y_test, svm_pred))

print('Confusion Matrix \n', confusion_matrix(y_test, svm_pred))
print('AUC is: {:.4f}'.format(roc_auc_score(y_test, svm_predprob)))

Training  model takes 12438.52 seconds
Accuracy of SVM on test set: 0.572
Predicting test data takes 229.889 seconds
Classification error rate: 0.4284185747913546
Classification report 
               precision    recall  f1-score   support

           0       0.54      0.98      0.69      4613
           1       0.90      0.17      0.29      4733

    accuracy                           0.57      9346
   macro avg       0.72      0.58      0.49      9346
weighted avg       0.72      0.57      0.49      9346

Confusion Matrix 
 [[4520   93]
 [3911  822]]
AUC is: 0.6612


In [11]:
#Save trained SVM model
pickle.dump(svm_best, open("C:/Users/liqia/Spring2021-Project5-group-1-1-master/output/best_svm.p",'wb'))
#Load SVM balanced model
pickle.load(open("C:/Users/liqia/Spring2021-Project3-group-3-master/output/best_svm.p",'rb'))

SVC(C=10, gamma=0.0001, probability=True)

weighted SVM

In [12]:
weighted_svm = SVC(gamma = 'scale', class_weight = 'balanced')

#CV Weighted SVM 
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(weighted_svm, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.3f' % np.mean(scores))

Mean ROC AUC: 0.665


In [13]:
balance = [{0:598.0, 1:2402.0},{0:1,1:100}, {0:1,1:10}, {0:1,1:1}, {0:10,1:1}, {0:100,1:1}]
param_grid = dict(class_weight=balance)

grid_weightedsvm = GridSearchCV(estimator=weighted_svm, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
grid_weightedsvm.fit(X,y)
grid_weightedsvm.best_params_
grid_weightedsvm.best_estimator_

SVC(class_weight={0: 598.0, 1: 2402.0})

In [14]:
grid_weightedsvm.best_params_
grid_weightedsvm.best_estimator_

SVC(class_weight={0: 598.0, 1: 2402.0})

In [15]:
print("Best: %f using %s" % (grid_weightedsvm.best_score_, grid_weightedsvm.best_params_))
# report all configurations
means = grid_weightedsvm.cv_results_['mean_test_score']
stds = grid_weightedsvm.cv_results_['std_test_score']
params = grid_weightedsvm.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.687006 using {'class_weight': {0: 598.0, 1: 2402.0}}
0.687006 (0.009648) with: {'class_weight': {0: 598.0, 1: 2402.0}}
0.488737 (0.014635) with: {'class_weight': {0: 1, 1: 100}}
0.646140 (0.010056) with: {'class_weight': {0: 1, 1: 10}}
0.566836 (0.028559) with: {'class_weight': {0: 1, 1: 1}}
0.553746 (0.029255) with: {'class_weight': {0: 10, 1: 1}}
0.541445 (0.028637) with: {'class_weight': {0: 100, 1: 1}}


In [16]:
weighted_svm_best = SVC(gamma = 'scale', class_weight ={0: 598.0, 1: 2402.0},probability=True)
start_time=time.time()
weighted_svm_best.fit(X, y)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

print('Accuracy of weighted SVM on test set: {:.3f}'.format(weighted_svm_best.score(X,y)))

start = time.time()
weighted_svm_pred = weighted_svm_best.predict(X_test)
end = time.time()
print(weighted_svm_pred[0:5,])

weighted_svm_predprob = weighted_svm_best.predict_proba(X_test)[:,1]

print("Predicting test data takes %s seconds" % round((end - start),3))
print('Classification error rate:', np.mean(np.array(y_test)!= weighted_svm_pred))
print('Classification report \n', classification_report(y_test, weighted_svm_pred))

print('Confusion Matrix \n', confusion_matrix(y_test, weighted_svm_pred))
print('AUC is: {:.4f}'.format(roc_auc_score(y_test, weighted_svm_predprob)))

Training  model takes 2278.569 seconds
Accuracy of weighted SVM on test set: 0.552
[0 1 1 0 1]
Predicting test data takes 52.606 seconds
Classification error rate: 0.33511662743419646
Classification report 
               precision    recall  f1-score   support

           0       0.76      0.46      0.58      4613
           1       0.62      0.86      0.72      4733

    accuracy                           0.66      9346
   macro avg       0.69      0.66      0.65      9346
weighted avg       0.69      0.66      0.65      9346

Confusion Matrix 
 [[2141 2472]
 [ 660 4073]]
AUC is: 0.7417


In [17]:
#Save trained weighted SVM model
pickle.dump(weighted_svm_best, open("C:/Users/liqia/Spring2021-Project5-group-1-1-master/output/best_weighted_svm.p",'wb'))

#Load weighted SVM model
pickle.load(open("C:/Users/liqia/Spring2021-Project5-group-1-1-master/output/best_weighted_svm.p",'rb'))

SVC(class_weight={0: 598.0, 1: 2402.0}, probability=True)